## Tests on preprocessing techniques

Tests were performed on 4-layer CNN model using different preprocessing techniques with 10 epochs including:
- Normalization
- Noise removal
- Image augmentation
- Image resizing
- Rectangle/square bounding box

In [0]:
#directly use raw data
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
    
        
        #without bounding box
        temp_images[i] = cv2.resize(train_images[img_idx],(28,28))
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)

#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")
y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")
final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

final_images = final_images.reshape(final_images.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization


X_train.shape
number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
# Fully connected layer

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator()
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

#model.fit(X_train, Y_train, batch_size=128, nb_epoch=10, validation_data=(X_test, Y_test))

#fit the data and valid
history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=10,validation_data=test_generator, validation_steps=4000//64)

#
score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
X_train original shape (35000, 28, 28)
y_train original shape (35000,)
X_test original shape (4000, 28, 28)
y_test original shape (4000,)
Epoch 1/10
546/546 [==============================] - 11s 19ms/step - loss: 2.3563 - acc: 0.1067 - val_loss: 2.3019 - val_acc: 0.1109
Epoch 2/10
546/546 [==============================] - 8s 15ms/step - loss: 2.3022 - acc: 0.1083 - val_loss: 2.3009 - val_acc: 0.1105
Epoch 3/10
546/546 [==============================] - 8s 15ms/step - loss: 2.3018 - acc: 0.1072 - val_loss: 2.3026 - val_acc: 0.1087
Epoch 4/10
546/546 [==============================] - 8s 15ms/step - loss: 2.3022 - acc: 0.1091 - val_loss: 2.3021 - val_acc: 0.1108
Epoch 5/10
546/546 [==============================] - 8s 15ms/step - loss: 2.3022 - acc: 0.1091 - val_loss: 2.3019 - val_acc: 0.1072
Epoch 6/10
546/546 [==============================] - 8s 15ms/ste

In [0]:
#applying normalization
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
    
        
        #without bounding box
        temp_images[i] = cv2.resize(train_images[img_idx],(28,28))
       
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)


# (X_train, y_train), (X_test, y_test) = mnist.load_data()



#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")
y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")
final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

final_images = final_images.reshape(final_images.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization
X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
# Fully connected layer

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))

# model.add(Convolution2D(10,3,3, border_mode='same'))
# model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator()
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

#model.fit(X_train, Y_train, batch_size=128, nb_epoch=10, validation_data=(X_test, Y_test))

#fit the data and valid
history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=10,validation_data=test_generator, validation_steps=4000//64)

#
score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
X_train original shape (35000, 28, 28)
y_train original shape (35000,)
X_test original shape (4000, 28, 28)
y_test original shape (4000,)
Epoch 1/10
546/546 [==============================] - 9s 17ms/step - loss: 2.3024 - acc: 0.1072 - val_loss: 2.3018 - val_acc: 0.1109
Epoch 2/10
546/546 [==============================] - 8s 15ms/step - loss: 2.3019 - acc: 0.1081 - val_loss: 2.3012 - val_acc: 0.1108
Epoch 3/10
546/546 [==============================] - 8s 15ms/step - loss: 2.3019 - acc: 0.1101 - val_loss: 2.3018 - val_acc: 0.1085
Epoch 4/10
546/546 [==============================] - 8s 15ms/step - loss: 2.3017 - acc: 0.1098 - val_loss: 2.3016 - val_acc: 0.1115
Epoch 5/10
546/546 [==============================] - 8s 15ms/step - loss: 2.3018 - acc: 0.1099 - val_loss: 2.3019 - val_acc: 0.1062
Epoch 6/10
546/546 [==============================] - 8s 15ms/step

In [0]:
#normalization+noise removal
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
        
        #reducing noise
        temp_images[i] = cv2.resize(threshed_img,(28,28))
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)


# (X_train, y_train), (X_test, y_test) = mnist.load_data()



#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")

y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")

final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

final_images = final_images.reshape(final_images.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization
X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
# Fully connected layer

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator()
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

#model.fit(X_train, Y_train, batch_size=128, nb_epoch=10, validation_data=(X_test, Y_test))

#fit the data and valid
history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=10,validation_data=test_generator, validation_steps=4000//64)

#
score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
X_train original shape (35000, 28, 28)
y_train original shape (35000,)
X_test original shape (4000, 28, 28)
y_test original shape (4000,)
Epoch 1/10
546/546 [==============================] - 9s 17ms/step - loss: 1.8204 - acc: 0.3279 - val_loss: 1.0498 - val_acc: 0.6565
Epoch 2/10
546/546 [==============================] - 8s 15ms/step - loss: 0.7235 - acc: 0.7731 - val_loss: 0.5260 - val_acc: 0.8366
Epoch 3/10
546/546 [==============================] - 8s 15ms/step - loss: 0.4549 - acc: 0.8594 - val_loss: 0.4158 - val_acc: 0.8704
Epoch 4/10
546/546 [==============================] - 8s 15ms/step - loss: 0.3512 - acc: 0.8915 - val_loss: 0.3795 - val_acc: 0.8803
Epoch 5/10
546/546 [==============================] - 8s 15ms/step - loss: 0.2833 - acc: 0.9102 - val_loss: 0.3489 - val_acc: 0.8920
Epoch 6/10
546/546 [==============================] - 8s 15ms/step

In [0]:
#normalization+noise removal+image augmentation
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
             
        
        #reducing noise
        temp_images[i] = cv2.resize(threshed_img,(28,28))
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)

#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")
y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")
final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

final_images = final_images.reshape(final_images.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization
X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Fully connected layer
BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

#model.fit(X_train, Y_train, batch_size=128, nb_epoch=10, validation_data=(X_test, Y_test))

#fit the data and valid
history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=10,validation_data=test_generator, validation_steps=4000//64)

#
score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
X_train original shape (35000, 28, 28)
y_train original shape (35000,)
X_test original shape (4000, 28, 28)
y_test original shape (4000,)
Epoch 1/10
546/546 [==============================] - 16s 30ms/step - loss: 2.0577 - acc: 0.2232 - val_loss: 1.4615 - val_acc: 0.4521
Epoch 2/10
546/546 [==============================] - 15s 28ms/step - loss: 1.1555 - acc: 0.6156 - val_loss: 0.7323 - val_acc: 0.7566
Epoch 3/10
546/546 [==============================] - 15s 28ms/step - loss: 0.7583 - acc: 0.7641 - val_loss: 0.5397 - val_acc: 0.8313
Epoch 4/10
546/546 [==============================] - 16s 28ms/step - loss: 0.6127 - acc: 0.8088 - val_loss: 0.3944 - val_acc: 0.8745
Epoch 5/10
546/546 [==============================] - 16s 28ms/step - loss: 0.5441 - acc: 0.8336 - val_loss: 0.3959 - val_acc: 0.8742
Epoch 6/10
546/546 [==============================] - 15s 28m

In [0]:
#normalization+noise removal+image augmentation+resizing
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]] 
        
        #reducing noise
        temp_images[i] = cv2.resize(threshed_img,(80,80))
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)



#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")
y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")
final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 80, 80

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 80, 80, 1)
X_test = X_test.reshape(X_test.shape[0], 80, 80, 1)

final_images = final_images.reshape(final_images.shape[0], 80, 80, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization
X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(80,80,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Fully connected layer
BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

#fit the data and valid
history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=30,validation_data=test_generator, validation_steps=4000//64)

#
score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
X_train original shape (35000, 80, 80)
y_train original shape (35000,)
X_test original shape (4000, 80, 80)
y_test original shape (4000,)
Epoch 1/30
546/546 [==============================] - 43s 78ms/step - loss: 2.0702 - acc: 0.2339 - val_loss: 1.6735 - val_acc: 0.4052
Epoch 2/30
546/546 [==============================] - 39s 72ms/step - loss: 1.5003 - acc: 0.4865 - val_loss: 1.1749 - val_acc: 0.6080
Epoch 3/30
546/546 [==============================] - 39s 71ms/step - loss: 1.1679 - acc: 0.6155 - val_loss: 0.9150 - val_acc: 0.7068
Epoch 4/30
546/546 [==============================] - 39s 71ms/step - loss: 0.9620 - acc: 0.6901 - val_loss: 0.7367 - val_acc: 0.7683
Epoch 5/30
546/546 [==============================] - 38s 70ms/step - loss: 0.8412 - acc: 0.7315 - val_loss: 0.7077 - val_acc: 0.7779
Epoch 6/30
546/546 [==============================] - 39s 71m

In [0]:
#normalization+noise removal+image augmentation+square bounding box
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if temp*temp > largest_area:
                largest_area = temp*temp
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
        
        #with bounding box
        temp_images[i] = cv2.resize(b,(28,28))
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)

#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")
y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")
final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

final_images = final_images.reshape(final_images.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization
X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Fully connected layer
BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

#fit the data and valid
history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=10,validation_data=test_generator, validation_steps=4000//64)

score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
X_train original shape (35000, 28, 28)
y_train original shape (35000,)
X_test original shape (4000, 28, 28)
y_test original shape (4000,)
Epoch 1/10
546/546 [==============================] - 17s 30ms/step - loss: 0.7855 - acc: 0.7597 - val_loss: 0.3645 - val_acc: 0.9052
Epoch 2/10
546/546 [==============================] - 15s 28ms/step - loss: 0.4286 - acc: 0.8855 - val_loss: 0.3020 - val_acc: 0.9337
Epoch 3/10
546/546 [==============================] - 15s 28ms/step - loss: 0.3626 - acc: 0.9056 - val_loss: 0.2935 - val_acc: 0.9372
Epoch 4/10
546/546 [==============================] - 15s 28ms/step - loss: 0.3307 - acc: 0.9146 - val_loss: 0.2765 - val_acc: 0.9411
Epoch 5/10
546/546 [==============================] - 16s 28ms/step - loss: 0.3160 - acc: 0.9191 - val_loss: 0.2488 - val_acc: 0.9466
Epoch 6/10
546/546 [==============================] - 16s 29m

In [0]:
#normalization+noise removal+image augmentation+rectangle bounding box
%reset -f
import torch
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
def get_data(images):
    train_images = images
    temp_images = []
    
    for i in range(len(train_images)):
        temp_images.append([])
        img_idx = i
        image = np.ascontiguousarray(train_images[img_idx], dtype=np.uint8)

        ret, threshed_img = cv2.threshold(image,
                        240, 255, cv2.THRESH_BINARY)
        img, contours, her = cv2.findContours(threshed_img, cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        largest_dim = {"x":0, "y":0, "w":0, "h":0}
        largest_area = 0
        for num in contours:
            x,y,w,h = cv2.boundingRect(num)
            
            temp = 0
            
            if w > h:
                temp=w
            else:
                temp=h
                
            if w*h > largest_area:
                largest_area = w*h
                largest_dim["x"] = x
                largest_dim["y"] = y
                largest_dim["w"] = w
                largest_dim["h"] = h
        

        b = threshed_img[largest_dim["y"]:largest_dim["y"]+largest_dim["h"],largest_dim["x"]:largest_dim["x"]+largest_dim["w"]]
        
        #with bounding box
        temp_images[i] = cv2.resize(b,(28,28))
    
    temp_images = np.asarray(temp_images, dtype=np.float32)
    
    
    return temp_images
  
def split_data(train_size, valid_size, test_size):
    images = pd.read_pickle('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_images.pkl')
    labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/ML/comp551_project_3/input/train_labels.csv')
    return (images[:train_size], images[train_size:train_size+valid_size],images[train_size+valid_size:]),(labels[:train_size], labels[train_size:train_size+valid_size],labels[train_size+valid_size:])



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(25)


#split data in train, validation, test
(X_train, X_test, final_images), (train_labels_new, test_labels_new, final_labels)=split_data(35000, 4000, 1000)

#pre processing it, resize it
X_train = get_data(X_train)
X_test = get_data(X_test)
final_images = get_data(final_images)

# do something to fit in kera model---------------------------------------------
y_train = np.array(train_labels_new["Category"].tolist(), dtype="uint8")
y_test = np.array(test_labels_new["Category"].tolist(), dtype="uint8")
final_labels = np.array(final_labels["Category"].tolist(), dtype="uint8")

print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)
print("X_test original shape", X_test.shape)
print("y_test original shape", y_test.shape)

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

final_images = final_images.reshape(final_images.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
final_images = final_images.astype('float32')

#normalization
X_train/=255
X_test/=255
final_images/=255

X_train.shape

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]
#------------------------------------------------------------------------------

#2d convolution----------------------------------------------------------------
# Three steps to Convolution
# 1. Convolution
# 2. Activation
# 3. Polling
# Repeat Steps 1,2,3 for adding more hidden layers
# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Fully connected layer
BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#----------------------------------------------------------------------------------------
#generate additional images

gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
#--------------------------------------------------------------------------------
test_gen = ImageDataGenerator()

train_generator = gen.flow(X_train, Y_train, batch_size=64)
test_generator = test_gen.flow(X_test, Y_test, batch_size=64)

#fit the data and valid
history = model.fit_generator(train_generator, steps_per_epoch=35000//64, epochs=10,validation_data=test_generator, validation_steps=4000//64)

score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
X_train original shape (35000, 28, 28)
y_train original shape (35000,)
X_test original shape (4000, 28, 28)
y_test original shape (4000,)
Epoch 1/10
546/546 [==============================] - 17s 31ms/step - loss: 0.9408 - acc: 0.7181 - val_loss: 0.5437 - val_acc: 0.8672
Epoch 2/10
546/546 [==============================] - 15s 28ms/step - loss: 0.6141 - acc: 0.8390 - val_loss: 0.4649 - val_acc: 0.8867
Epoch 3/10
546/546 [==============================] - 16s 28ms/step - loss: 0.5552 - acc: 0.8582 - val_loss: 0.4425 - val_acc: 0.8905
Epoch 4/10
546/546 [==============================] - 15s 28ms/step - loss: 0.5214 - acc: 0.8677 - val_loss: 0.4277 - val_acc: 0.8928
Epoch 5/10
546/546 [==============================] - 16s 29ms/step - loss: 0.4956 - acc: 0.8741 - val_loss: 0.4284 - val_acc: 0.8935
Epoch 6/10
546/546 [==============================] - 16s 29m